In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import TruncatedSVD
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.impute import SimpleImputer
from tensorflow.keras.layers import Add, LeakyReLU
from tensorflow.keras.activations import relu
seed = 1

In [7]:
############################# Step 1: Data Preprocessing ############################# 

# Define directories for training and validation data
train_dir = "./altis_power_cap_res/2_dual_cap/train"
validation_dir = "./ecp_power_cap_res/2_dual_cap/validation"
save_model_dir = "./altis_power_cap_res/2_dual_cap/model"
os.makedirs(save_model_dir, exist_ok=True)

# Load CSV files from train directory
train_csv_files = [f for f in os.listdir(train_dir) if f.endswith(".csv")]
validation_csv_files = [f for f in os.listdir(validation_dir) if f.endswith(".csv")]

# Load and merge training data while filtering out specified CPU power caps
train_data = []
for file in train_csv_files:
    file_path = os.path.join(train_dir, file)
    df = pd.read_csv(file_path)

    # Drop rows where CPU Power Cap is 120 or 130
    df = df[~df["CPU Power Cap"].isin([120, 130])]

    train_data.append(df)

# Merge all training data
train_df = pd.concat(train_data, ignore_index=True)

# Load validation data while filtering out specified CPU power caps
validation_data = {}
for file in validation_csv_files:
    file_path = os.path.join(validation_dir, file)
    df = pd.read_csv(file_path)

    # Drop rows where CPU Power Cap is 120 or 130
    df = df[~df["CPU Power Cap"].isin([120, 130])]

    validation_data[file] = df

# Drop any rows with missing values
train_df.dropna(inplace=True)
for key in validation_data:
    validation_data[key].dropna(inplace=True)

In [8]:
############################# Step 2: Build MLP Model ############################# 

# Define feature columns and target column
feature_cols = ["CPU Power Cap", "GPU Power Cap", "IPS", "Memory Throughput", "SM Clock", "DRAM Active", "FP Active"]
feature_cols = ["CPU Power Cap", "GPU Power Cap", "Memory Throughput", "SM Clock", "DRAM Active", "FP Active"]
# feature_cols = ["CPU Power Cap", "GPU Power Cap", "Memory Throughput", "SM Clock"]

target_col = "Performance"

# Extract features and target for training
X_train = train_df[feature_cols].values
y_train = train_df[target_col].values.reshape(-1, 1)

# Normalize features and target
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)

# Build the improved MLP model
# Define the MLP model properly
model = Sequential([
    tf.keras.Input(shape=(X_train_scaled.shape[1],)),  # Explicitly define input shape
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='selu'),
    Dense(32, activation='selu'),
    Dense(1, activation='linear')
])

In [28]:
############################# Step 3: Train MLP Model #############################
# Compile the model with a different optimizer
model.compile(optimizer='nadam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train_scaled, y_train_scaled, epochs=50, batch_size=32, verbose=3)

# Save the trained model
model.save(os.path.join(save_model_dir, "performance_prediction_model.h5"))

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50


In [31]:
############################# Step 4: Evaluate Model Performance ############################# 

# Define model path and load trained model
model_path = os.path.join(save_model_dir, "performance_prediction_model.h5")  # Legacy format
model = tf.keras.models.load_model(model_path, custom_objects={"mse": tf.keras.losses.MeanSquaredError()})

# Define feature columns and target column
feature_cols = ["CPU Power Cap", "GPU Power Cap", "IPS", "Memory Throughput", "SM Clock", "DRAM Active", "FP Active"]
feature_cols = ["CPU Power Cap", "GPU Power Cap", "Memory Throughput", "SM Clock", "DRAM Active", "FP Active"]
# feature_cols = ["CPU Power Cap", "GPU Power Cap", "Memory Throughput", "SM Clock"]
target_col = "Performance"

# Iterate over validation CSV files
for file in validation_csv_files:
    file_path = os.path.join(validation_dir, file)
    new_app_name = file.replace("_performance.csv", "")

    # Load validation data
    df_new = pd.read_csv(file_path)
    df_new["Power Pair"] = list(zip(df_new["CPU Power Cap"], df_new["GPU Power Cap"]))

    # Sample 20% of power pairs
    df_sampled = df_new.sample(frac=0.2, random_state=seed)  # Select 20% of rows
    sampled_pairs = df_sampled["Power Pair"].unique()

    true_values, nn_predicted_values, accuracy_values = [], [], []

    for power_pair in sampled_pairs:
        # Extract feature values for the selected power pair
        X_sample = df_new[df_new["Power Pair"] == power_pair][feature_cols].values

        # Normalize features
        X_sample_scaled = scaler_X.transform(X_sample)

        # Predict performance
        y_pred = model.predict(X_sample_scaled)
        predicted_value = scaler_y.inverse_transform(y_pred)[0][0]

        # Store true and predicted values
        true_value = df_new.loc[df_new["Power Pair"] == power_pair, "Performance"].values[0]
        true_values.append(true_value)
        nn_predicted_values.append(predicted_value)

        # Compute percentage-based prediction accuracy
        accuracy = 100 - (abs(true_value - predicted_value) / true_value * 100)
        accuracy_values.append(accuracy)

    # Compute evaluation metrics
    nn_mae = mean_absolute_error(true_values, nn_predicted_values)
    nn_rmse = np.sqrt(mean_squared_error(true_values, nn_predicted_values))
    nn_r2 = r2_score(true_values, nn_predicted_values)
    avg_accuracy = np.mean(accuracy_values)

    # Print evaluation results
    print(f"Neural Network Prediction for {new_app_name} (Sampled 20% Power Pairs, One-by-One Prediction):")
    print(f"MAE: {nn_mae:.4f}")
    print(f"Prediction Accuracy: {avg_accuracy:.2f}%\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Neural Network Prediction for lammps (Sampled 20% Power Pairs, One-by-One Prediction):
MAE: 0.0274
Prediction Accuracy: 96.66%

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/st

In [9]:
############################# Step 5: Populate Initial Performance Matrix ############################# 

model_path = os.path.join(save_model_dir, "performance_prediction_model.h5")  # Legacy format
model = tf.keras.models.load_model(model_path, custom_objects={"mse": tf.keras.losses.MeanSquaredError()})
# feature_cols = ["CPU Power Cap", "GPU Power Cap", "IPS", "Memory Throughput", "SM Clock", "DRAM Active", "FP Active"]
feature_cols = ["CPU Power Cap", "GPU Power Cap", "Memory Throughput", "SM Clock", "DRAM Active", "FP Active"]
# feature_cols = ["CPU Power Cap", "GPU Power Cap", "Memory Throughput", "SM Clock"]
target_col = "Performance"

# Dictionary to store application data
app_data = {}

# Process each CSV file (each application)
for file in train_csv_files:
    file_path = os.path.join(train_dir, file)
    app_name = file.replace("_performance.csv", "")
    
    df = pd.read_csv(file_path)
    required_columns = ["CPU Power Cap", "GPU Power Cap", "Performance"]
    df = df[required_columns]

    # Create a unique power pair column
    df["Power Pair"] = list(zip(df["CPU Power Cap"], df["GPU Power Cap"]))
    
    # Store application data
    app_data[app_name] = df[["Power Pair", "Performance"]].set_index("Power Pair")

# Combine all applications into a single 2D matrix
performance_matrix = pd.DataFrame(index=sorted(set().union(*[df.index for df in app_data.values()])),
                                  columns=sorted(app_data.keys()))


# Populate the performance matrix with training data
for app_name, df in app_data.items():
    performance_matrix[app_name] = df["Performance"]

In [10]:
############################# Step 6: Construct Sparse Matrix and Predict Missing Value ############################# 

# Directories
save_model_dir = "./altis_power_cap_res/2_dual_cap/model"
os.makedirs(save_model_dir, exist_ok=True)

# Results storage
nn_results = []
cf_results = []

# Process each new application
for file in validation_csv_files:
    file_path = os.path.join(validation_dir, file)
    new_app_name = file.replace("_performance.csv", "")

    # Load validation data
    df_new = pd.read_csv(file_path)
    df_new["Power Pair"] = list(zip(df_new["CPU Power Cap"], df_new["GPU Power Cap"]))

    # Add new application to performance matrix
    performance_matrix[new_app_name] = np.nan

    #############  Predict 20% of power pairs using trained NN --- Construct Sparse Matrix  ############# 
    
    df_sampled = df_new.sample(frac=0.2, random_state=seed)  # Select 20% of rows
    sampled_pairs = df_sampled["Power Pair"].unique()

    true_values, nn_predicted_values = [], []
    for power_pair in sampled_pairs:
        X_sample = df_new[df_new["Power Pair"] == power_pair][feature_cols].values
        X_sample_scaled = scaler_X.transform(X_sample)
        
        # Predict performance
        y_pred = model.predict(X_sample_scaled)
        predicted_value = scaler_y.inverse_transform(y_pred)[0][0]
    
        # Fill performance matrix with NN predictions
        performance_matrix.at[power_pair, new_app_name] = predicted_value

        # Store true and predicted values for error calculation
        if power_pair in df_new.set_index("Power Pair").index:
            actual = df_new.loc[df_new["Power Pair"] == power_pair, "Performance"].values[0]
            true_values.append(actual)
            nn_predicted_values.append(predicted_value)
    
    ############## Compute NN Prediction Accuracy ##############
    
    nn_mae = mean_absolute_error(true_values, nn_predicted_values)
    nn_rmse = np.sqrt(mean_squared_error(true_values, nn_predicted_values))
    nn_r2 = r2_score(true_values, nn_predicted_values)

    # Percentage-Based Prediction Error
    nn_pred_error = np.mean(np.abs((np.array(true_values) - np.array(nn_predicted_values)) / np.array(true_values))) * 100

    nn_results.append((new_app_name, nn_mae, nn_rmse, nn_r2, nn_pred_error))

    print(f"NN Prediction for {new_app_name}: MAE={nn_mae:.4f}, Prediction Accuracy={100 - nn_pred_error:.2f}%")

    ##############  Train Neural CF  ##############
    power_pair_map = {pair: i for i, pair in enumerate(performance_matrix.index)}
    app_map = {app: i for i, app in enumerate(performance_matrix.columns)}

    train_data = []
    train_labels = []
    
    for app in performance_matrix.columns:
        for power_pair in performance_matrix.index:
            if not np.isnan(performance_matrix.at[power_pair, app]):
                train_data.append([power_pair_map[power_pair], app_map[app]])
                train_labels.append(performance_matrix.at[power_pair, app])

    train_data = np.array(train_data)
    train_labels = np.array(train_labels)

    # Define Neural CF Model
    num_power_pairs = len(power_pair_map)
    num_apps = len(app_map)
    latent_dim = 10  # Embedding size

    input_power_pair = Input(shape=(1,))
    input_app = Input(shape=(1,))

    power_embedding = Embedding(num_power_pairs, latent_dim)(input_power_pair)
    app_embedding = Embedding(num_apps, latent_dim)(input_app)

    power_vec = Flatten()(power_embedding)
    app_vec = Flatten()(app_embedding)

    merged = Concatenate()([power_vec, app_vec])
    dense_1 = Dense(128, activation='selu')(merged)
    dense_2 = Dense(64, activation='selu')(dense_1)
    dense_3 = Dense(32, activation='selu')(dense_2)
    output = Dense(1, activation='linear')(dense_3)

    ncf_model = Model(inputs=[input_power_pair, input_app], outputs=output)
    ncf_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    ncf_model.fit([train_data[:, 0], train_data[:, 1]], train_labels, epochs=25, batch_size=32, verbose=3)

    ##############  Predict Missing Value via NCF  ##############
    for power_pair in performance_matrix.index:
        if np.isnan(performance_matrix.at[power_pair, new_app_name]):
            power_idx = power_pair_map[power_pair]
            app_idx = app_map[new_app_name]
            pred_value = ncf_model.predict([np.array([power_idx]), np.array([app_idx])])[0][0]
            performance_matrix.at[power_pair, new_app_name] = pred_value

    ############### Compute CF Prediction Accuracy ##############
    true_values = df_new.set_index("Power Pair")["Performance"]
    predicted_values = performance_matrix[new_app_name].reindex(true_values.index)

    cf_mae = mean_absolute_error(true_values, predicted_values)
    cf_rmse = np.sqrt(mean_squared_error(true_values, predicted_values))
    cf_r2 = r2_score(true_values, predicted_values)

    # Percentage-Based Prediction Error
    cf_pred_error = np.mean(np.abs((true_values - predicted_values) / true_values)) * 100

    cf_results.append((new_app_name, cf_mae, cf_rmse, cf_r2, cf_pred_error))

    print(f"NCF Prediction for {new_app_name}: MAE={cf_mae:.4f}, Prediction Accuracy={100 - cf_pred_error:.2f}%")

# Convert results into a DataFrame and display
nn_results_df = pd.DataFrame(nn_results, columns=["Application", "MAE", "RMSE", "R²", "Prediction Error (%)"])
cf_results_df = pd.DataFrame(cf_results, columns=["Application", "MAE", "RMSE", "R²", "Prediction Error (%)"])

I0000 00:00:1740696941.145556  815448 service.cc:148] XLA service 0x7fbcb80040f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740696941.145597  815448 service.cc:156]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1740696941.180292  815448 cuda_dnn.cc:529] Loaded cuDNN version 90701


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


I0000 00:00:1740696941.375841  815448 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
NN Prediction for sw4lite: MAE=0.0353, Prediction Accuracy=95.92%
Epoch 1/25
Epoch 2/25
Epoch 3/25
Epoch 4/25
Epoch 5/25
Epoch 6/25
Epoch 7/25
Epoch 8/25
Epoch 9/25
Epoch 10/25
Epoch 11/25
Epoch 12/25
Epoch 13/25
Epoch 14/25
Epoch 15/25
Epoch 16/25
Epoch 17/25
Epoch 18/25
Epoch 19/25
Epoch 20/25
Epoch 21/25
Epoch 2

In [11]:
# Print stored accuracy metrics after all iterations
print("\n=== Neural Network Prediction Accuracy ===")
for app, mae, _, _, pred_error in nn_results:
    print(f"{app}: MAE={mae:.4f}, Prediction Accuracy={100 - pred_error:.2f}%")

print("\n=== Neural Collaborative Filtering Prediction Accuracy ===")
for app, mae, _, _, pred_error in cf_results:
    print(f"{app}: MAE={mae:.4f}, Prediction Accuracy={100 - pred_error:.2f}%")


=== Neural Network Prediction Accuracy ===
sw4lite: MAE=0.0353, Prediction Accuracy=95.92%

=== Neural Collaborative Filtering Prediction Accuracy ===
sw4lite: MAE=0.0308, Prediction Accuracy=96.31%


In [10]:
performance_matrix

,bfs,cfd,cfd_double,fdtd2d,gemm,gups,kmeans,lavamd,maxflops,nw,...,Resnet50,sw4lite,Laghos,NAMD,miniGAN,gromacs,bert_large,UNet,CRADL,XSBench
"(120, 150)",0.637024,0.637359,0.818064,0.733109,0.638716,0.970849,0.899788,0.656863,0.716044,0.615326,...,0.748207,0.593205,0.645191,0.620911,0.679260,0.684636,0.806175,0.686133,0.640825,0.581656
"(120, 160)",0.601343,0.630359,0.843605,0.785478,0.636686,0.985136,0.899804,0.671545,0.753145,0.634912,...,0.763345,0.579719,0.640847,0.611251,0.696895,0.690649,0.823847,0.692480,0.634400,0.594419
"(120, 170)",0.637012,0.630417,0.843523,0.845871,0.642886,0.985148,0.918240,0.656853,0.773292,0.624922,...,0.756815,0.594429,0.642583,0.613256,0.725310,0.703670,0.829256,0.704401,0.640516,0.590241
"(120, 180)",0.632312,0.630321,0.870889,0.879836,0.644969,0.985232,0.918253,0.633694,0.743598,0.615301,...,0.761961,0.588311,0.640753,0.590209,0.728197,0.712597,0.847633,0.714047,0.640477,0.570782
"(120, 190)",0.637010,0.630438,0.870746,0.916513,0.642902,0.999798,0.918217,0.666623,0.743510,0.624946,...,0.757247,0.599265,0.646845,0.587302,0.741272,0.717044,0.858999,0.712057,0.634252,0.559531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(200, 210)",0.988398,0.983083,0.964160,0.999803,0.970615,0.999860,0.999771,0.999971,1.000000,0.999959,...,0.984617,0.986726,1.012247,0.991432,0.990818,0.984795,0.977278,0.963588,0.982609,0.978682
"(200, 220)",0.999967,0.983148,0.964155,0.999816,0.990035,0.999869,0.999768,0.978321,0.983114,0.975661,...,1.002311,0.945251,0.983839,0.953184,0.972923,0.968834,0.973787,0.963600,0.962764,0.927079
"(200, 230)",0.999982,1.000000,0.964168,0.999852,0.995034,0.999899,1.000000,0.999954,0.966676,0.975567,...,0.997511,0.922074,0.982641,0.952342,0.973780,0.964614,0.971409,0.946061,0.948404,0.915300
"(200, 240)",0.999965,0.983124,0.999882,0.999814,0.989992,0.999878,0.999878,0.988977,0.983013,0.869468,...,0.986865,0.953991,1.009568,0.992875,0.950191,0.962752,0.976246,0.978114,0.947420,0.981660


In [66]:
performance_matrix.to_csv("./prediction_res/performance_matrix.csv", index=True)